In [29]:
##m <- c("jpeg", "nlopt", "assertthat",
##"base64enc", "BH", "bitops", "car",
##"caret", "chron", "colorspace",
##"data.table", "DBI", "dichromat", "dplyr",
##"foreach", "formatR", "ggally”, ”ggplot2",
##"glmnet", "gtable", "hexbin", "highr",
##"htmltools", "htmlwidgets", "httpuv",
##"iterators", "knitr", "labeling",
##"lazyeval", "maps", "markdown",
##"matrixmodels", "minqa", "munsell",
##"nloptr", "pbkrtest", "plyr", "proto",
##"quantmod", "quantreg",
##"randomforest", "rbokeh", "rcolorbrewer",
##"rcpp", "rcppeigen", "repr", "reshape2",
##"rjsonio", "rmarkdown", "scales", "shiny",
##"sparsem", "tidyr", "ttr", "xtable",
##"xts", "yaml", "zoo", "ggally")
##install.packages(m)

## Instale y cargue las siguientes librerias
library(caret) 
library(glmnet)

#Load the data
train <- read.csv("train.csv")

#Column types
str(train)

table(train$Survived)
table(train$Pclass)
#table(train$Name)
table(train$Sex)
#table(train$Age)
table(train$SibSp)
table(train$Parch)
#table(train$Ticket)
#table(train$Fare)
#table(train$Cabin)
table(train$Embarked)


'data.frame':	891 obs. of  12 variables:
 $ PassengerId: int  1 2 3 4 5 6 7 8 9 10 ...
 $ Survived   : int  0 1 1 1 0 0 0 0 1 1 ...
 $ Pclass     : int  3 1 3 1 3 3 1 3 3 2 ...
 $ Name       : Factor w/ 891 levels "Abbing, Mr. Anthony",..: 109 191 358 277 16 559 520 629 417 581 ...
 $ Sex        : Factor w/ 2 levels "female","male": 2 1 1 1 2 2 2 2 1 1 ...
 $ Age        : num  22 38 26 35 35 NA 54 2 27 14 ...
 $ SibSp      : int  1 1 0 1 0 0 0 3 0 1 ...
 $ Parch      : int  0 0 0 0 0 0 0 1 2 0 ...
 $ Ticket     : Factor w/ 681 levels "110152","110413",..: 524 597 670 50 473 276 86 396 345 133 ...
 $ Fare       : num  7.25 71.28 7.92 53.1 8.05 ...
 $ Cabin      : Factor w/ 148 levels "","A10","A14",..: 1 83 1 57 1 1 131 1 1 1 ...
 $ Embarked   : Factor w/ 4 levels "","C","Q","S": 4 2 4 4 4 3 4 4 4 2 ...



  0   1 
549 342 


  1   2   3 
216 184 491 


female   male 
   314    577 


  0   1   2   3   4   5   8 
608 209  28  16  18   5   7 


  0   1   2   3   4   5   6 
678 118  80   5   4   5   1 


      C   Q   S 
  2 168  77 644 

In [2]:
#First 5 records
head(train)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,,S
6,0,3,"Moran, Mr. James",male,NA,0,0,330877,8.4583,,Q


In [4]:
## Cut data for training and testing (70% and 30%)
set.seed(987954)      # seed

## Partición de los datos

survivor_sampling_vector <- createDataPartition(train$Survived,  # output data
                                             p = 0.7,      # Percentage of data partition
                                             list = FALSE)

In [39]:
## 624 observations for training

survivor_train        <- train[survivor_sampling_vector,]
survivor_train_labels <- train$Survived[survivor_sampling_vector]          # training output variable
#str(survivor_train)

## 267 observaciones para prueba
survivor_test         <- train[-survivor_sampling_vector,]                 # Take the data that were not selected for training
survivo_test_labels  <- train$Survived[-survivor_sampling_vector]          # Test output variable 
#str(survivor_test)

In [40]:
## Logistic regression model

survival_model <- glm(Survived ~ Sex+Pclass+Sex+SibSp,                          # Variable 'output' vs all variables
                   data = train,                             # Data
                   family = binomial("logit"))               # Type of model


In [41]:
## Summary of the adjustment
summary(survival_model)


Call:
glm(formula = Survived ~ Sex + Pclass + Sex + SibSp, family = binomial("logit"), 
    data = train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.2689  -0.6735  -0.4747   0.6189   2.5148  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  3.43357    0.30543  11.242  < 2e-16 ***
Sexmale     -2.74314    0.19027 -14.417  < 2e-16 ***
Pclass      -0.93896    0.10647  -8.819  < 2e-16 ***
SibSp       -0.24812    0.09453  -2.625  0.00867 ** 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 1186.66  on 890  degrees of freedom
Residual deviance:  819.32  on 887  degrees of freedom
AIC: 827.32

Number of Fisher Scoring iterations: 4


In [42]:
## Model performance

train_predictions       <- predict(survival_model,                      # Modelo obtenido 
                                     newdata = survivor_train,          # Datos de entrenamiento
                                       type = "response")

train_class_predictions <- as.numeric(train_predictions > 0.5)       # Valores mayores a 0.5

mean(train_class_predictions == survivor_train$Survived)                  # Metrica de precision con la media

[1] 0.7932692

In [43]:
## performance test of the model

test_predictions       <-  predict(survival_model,                      # Modelo
                                    newdata = survivor_test,            # COnjunto de datos de prueba
                                     type = "response")

test_class_predictions <-  as.numeric(test_predictions > 0.5)        # Valores predichos mayores a 0.5

mean(test_class_predictions == survivor_test$Survived)                    # Precision

[1] 0.8164794

In [44]:
#https://triangleinequality.wordpress.com/2013/09/08/basic-feature-engineering-with-the-titanic-data/
#https://www.kaggle.com/c/titanic/data


In [56]:
#Load the data
test_kaggle <- read.csv("test.csv")

## Forecasting performance of the model
#head(test_kaggle)
test_kaggle_predictions       <-  predict(survival_model,                      # Modelo
                                    newdata = test_kaggle,            # COnjunto de datos de prueba
                                     type = "response")

test_kaggle_class_predictions <-  as.numeric(test_kaggle_predictions > 0.5)        # Valores predichos mayores a 0.5
test_kaggle$Survived<-test_kaggle_class_predictions
#mean(test_class_predictions == survivor_test$Survived)                    # Precision
test_kaggle_output <- test_kaggle[,c('PassengerId','Survived')]
#apply(test_kaggle, 
#test_kaggle_output
#            function(x) x$PassengerId+","+x$Survived)
#test_kaggle_output
#                test_kaggle[,c('Species','Survived')]
## lo escribe con formato csv.
write.csv(x = test_kaggle_output, 
          file = 'test_kaggle_output.csv',  # archivo
          row.names = FALSE)        # imprime los nombres de las filas?

PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,,S
897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,,S
